<a href="https://colab.research.google.com/github/ZainAli24/QUIZ-PREPARATION---OpenAI-Agents-SDK/blob/main/Class04_QUIZ_PREPARATION_Tools_and_associated_Tool_Settings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Tool Behaviour:**

Aapka sawal bilkul samajh mein aa gaya hai. Aap pooch rahe hain ke agar aap `tool_use_behavior` ko `"stop_on_first_tool"` set karte hain, aur agent ke paas multiple tools hain, toh kya un mein se koi bhi ek tool call hone par uska output final output ban jayega, aur agent aage kaam karna band kar dega? Main iska jawab aapke query aur documentation ke base par deta hoon.

---

### **`tool_use_behavior` Kya Hota Hai?**
`tool_use_behavior` ek setting hai jo yeh control karti hai ke agent tools ko kaise handle karega. Documentation ke mutabiq, iske kuch possible values hain:

- **`"run_llm_again"`** (default): Tool call hone ke baad uska result LLM (language model) ko diya jata hai, aur LLM phir se process karke response deta hai.
- **`"stop_on_first_tool"`**: Jab pehla tool call hota hai, agent ka execution wahin ruk jata hai, aur us tool ka output final output ban jata hai. LLM is output ko process nahi karta.
- **List of tool names**: Agar aap specific tools ki list dete hain, toh unmein se koi bhi tool call hone par agent ruk jata hai, aur us tool ka output final output hota hai.
- **Function**: Ek custom function jo tool results ke base par final output decide karta hai.

Aapka sawal `"stop_on_first_tool"` ke bare mein hai, toh main is par focus karta hoon.

---

### **`"stop_on_first_tool"` Ka Matlab**
Agar aap `tool_use_behavior="stop_on_first_tool"` set karte hain, toh yeh hota hai:

- **Pehla Tool Call**: Jaise hi agent koi bhi ek tool call karta hai, agent ka kaam wahin ruk jata hai.
- **Final Output**: Us tool ka output (jo result usne diya) directly final output ban jata hai.
- **LLM Ka Role**: Tool ke output ke baad LLM ko dobara call nahi kiya jata, yani LLM us result ko process nahi karta.

**Documentation Se Confirmation**:
> *"The output of the first tool call is used as the final output. This means that the LLM does not process the result of the tool call."*

Yani, pehla tool call hone ke baad agent aage nahi chalta, aur jo output us tool se milta hai, wahi final jawab hota hai.

---

### **Multiple Tools Ke Saath Kya Hoga?**
Agar agent ke paas multiple tools hain (jaise `fetch_weather`, `book_flight`, etc.), aur aapne `tool_use_behavior="stop_on_first_tool"` set kiya hai, toh:

- **Pehla Tool Hi Count Hota Hai**: Agent jab bhi pehla tool call karega (chahe woh multiple tools mein se koi bhi ho), uska output final output ban jayega.
- **Agent Ruk Jata Hai**: Pehle tool ke call hone ke baad agent aage processing nahi karta, chahe uske paas aur tools kyun na hon.
- **Example**:
  - Maan lijiye agent ko input milta hai: "Mausam check karo aur flight book karo."
  - Agar agent pehle `fetch_weather` tool call karta hai, toh uska output (jaise "Mausam cloudy hai, 25°C") final output ban jayega.
  - `book_flight` tool call nahi hoga, kyunki agent pehle tool ke baad ruk jata hai.

**Key Point**: Chahe agent ke paas kitne bhi tools hon, `"stop_on_first_tool"` mein pehla tool jo call hota hai, uska output final banega, aur baaki tools ignore ho jayenge.

---

### **Kaise Kaam Karta Hai Yeh Practically?**
Aap agent ko is tarah se configure kar sakte hain:

```python
agent = Agent[MyCustomContext](
    name="Assistant Agent",
    instructions="...",
    model=model,
    tools=[fetch_weather, book_flight],
    tool_use_behavior="stop_on_first_tool"  # Yeh setting
)
```

Is setting ke saath:
- Agar agent `fetch_weather` call karta hai, toh weather ka result final output hoga.
- Agar agent `book_flight` call karta hai, toh flight booking ka result final output hoga.
- Pehla tool jo bhi call hota hai, uske baad agent stop ho jata hai.

---

### **Agar Ek Turn Mein Multiple Tools Call Hon?**
Agar agent ek turn mein multiple tools call karne ki koshish karta hai, toh `"stop_on_first_tool"` ke case mein pehla tool jo call hota hai, uska output final output banega, aur baaki tools call nahi honge. Lekin yeh exact behavior SDK ke implementation par depend karta hai, aur documentation mein iske bare mein explicitly nahi kaha gaya. Aam taur par, pehla tool call hone ke baad agent ruk jata hai.

---

### **Jawab**
Haan, agar aap `tool_use_behavior` ko `"stop_on_first_tool"` set karte hain, aur agent ke paas multiple tools hain, toh un mein se koi bhi ek tool call hone par uska output final output ban jayega. Agent uske baad aage processing nahi karega, aur LLM tool ke result ko process nahi karega. Yeh behavior tab kaam aata hai jab aap chahte hain ke pehle tool ka result hi final jawab ho, bina kisi extra processing ke.

----------------------------

# **2. Function return final output confusion in case of stop_first_tool or stop_at_tool_names:**

Aapka sawal clear hai. Aap pooch rahe hain ke jab aap `tool_use_behavior=StopAtTools(stop_at_tool_names=["add"])` set karte hain, aur `add` function tool ke andar ek `print` statement hai, toh kya `add` function ka return value (`a + b - 5`) final output banega, ya `print` statement bhi final output ka hissa hoga? Main isko aapke code aur OpenAI Agents SDK ke documentation ([https://openai.github.io/openai-agents-python/ref/agent/#agents.agent.Agent.tool_use_behavior](https://openai.github.io/openai-agents-python/ref/agent/#agents.agent.Agent.tool_use_behavior)) ke base par simple tareeke se samjhaunga.

---

### **1. `tool_use_behavior=StopAtTools` Ka Matlab**
Documentation ke mutabiq, jab aap `tool_use_behavior` ko `StopAtTools(stop_at_tool_names=["add"])` set karte hain:
- Agent execution ruk jata hai jab specified tool (yani `"add"`) call hota hai.
- Us tool ka output (jo function return karta hai) directly final output ban jata hai.
- LLM tool ke result ko process nahi karta.

Documentation se quote:
> *"The agent will stop running if any of the tools in the list are called. The final output will be the output of the first matching tool call. The LLM does not process the result of the tool call."*

Yani, `add` tool ke return value ko final output mana jayega.

---

### **2. Aapka `add` Function**
Aapka `add` function aisa hai:

```python
@function_tool
def add(a: int, b: int) -> int:
    """Add Two Numbers."""
    print("---------Tool Called Sucessfully-----------")
    return a + b - 5
```

- **Function Ka Behavior**:
  - Yeh function do integers (`a`, `b`) leta hai.
  - Ek `print` statement chalata hai: `"---------Tool Called Sucessfully-----------"`.
  - Phir `a + b - 5` return karta hai, jo `int` type ka hota hai.

- **Return Value**: `return a + b - 5` hi function ka actual output hai, jo SDK ke liye final output banega jab `add` tool call hoga.
- **Print Statement**: `print` statement sirf console mein output display karta hai (side effect ke taur par). Yeh function ke return value ka hissa nahi hota.

---

### **3. Kya `print` Statement Final Output Ka Hissa Hai?**
**Nahi, `print` statement final output ka hissa nahi hai.**

- Jab aap `tool_use_behavior=StopAtTools(stop_at_tool_names=["add"])` use karte hain:
  - SDK `add` function ke **return value** (`a + b - 5`) ko final output ke taur par leta hai.
  - `print` statement jo console mein output karta hai (`---------Tool Called Sucessfully-----------`), woh final output ka hissa nahi banega. Yeh sirf debugging ya logging ke liye hota hai.

- **Example**:
  Aapka input hai: `"tell me whats the weather in karachi and What is 2 + 2 ?"`.
  - Agent pehle `add` tool call karega kyunki `stop_at_tool_names=["add"]` set hai aur input mein "2 + 2" ka zikr hai.
  - `add(2, 2)` call hoga:
    - `print` statement console mein yeh print karega: `---------Tool Called Sucessfully-----------`
    - Function return karega: `2 + 2 - 5 = -1`
  - Yeh `-1` final output banega, kyunki `add` tool ka return value yahi hai.
  - `fetch_weather` tool call nahi hoga, kyunki `stop_at_tool_names=["add"]` agent ko `add` tool call hone ke baad rok deta hai.

**Output**:
```python
---------Tool Called Sucessfully-----------
-1  # Yeh result.final_output se print hoga
```

---

### **4. Kaise Handle Hota Hai?**
- **SDK Ka Logic**:
  - Jab `Runner.run_sync` chalata hai, agent input ko process karta hai aur `add` tool ko call karta hai kyunki `stop_at_tool_names=["add"]` set hai.
  - `add` function ka return value (`int`) directly `result.final_output` ban jata hai, jaisa documentation mein kaha gaya hai.
  - `print` statement ka output console mein dikhta hai, lekin yeh `result.final_output` ka hissa nahi hota.

- **Documentation Se Confirmation**:
  > *"The final output will be the output of the first matching tool call."*
  Yeh clearly kehta hai ke tool ka return value final output hai, na ke koi side effects (jaise `print` statements).

---

### **Summary**
- **`add` Function Ka Return Value**: Jab `tool_use_behavior=StopAtTools(stop_at_tool_names=["add"])` set hai, `add` function ka return value (`a + b - 5`, yaani `-1` for `2 + 2`) final output banega.
- **`print` Statement**: Yeh console mein display hota hai, lekin final output (`result.final_output`) ka hissa nahi hai.
- **Behavior**: `add` tool call hone ke baad agent ruk jata hai, aur uska return value (`-1`) directly `result.final_output` ban jata hai.

---------------------------------------------------------------------------------------------------------------------------------------------------------------------

# **3. StopAtTools (TypedDict) confusion in Tool_Use_Behaviour:**

### **1. `StopAtTools` Class Kya Hai?**
Aapke di gayi definition ke mutabiq:

```python
class StopAtTools(TypedDict):
    stop_at_tool_names: list[str]
    """A list of tool names, any of which will stop the agent from running further."""
```

- **`StopAtTools`** ek `TypedDict` hai, jo Python ke `typing` module se aata hai. Yeh ek special type ka dictionary hai jo type hints ke liye use hota hai, taake aap specify kar saken ke dictionary mein kaunse keys aur unke types honge (yahan `stop_at_tool_names` ek `list[str]` hai).
- `TypedDict` **runtime** mein ek regular dictionary ke jaisa behave karta hai, lekin static type checkers (jaise `mypy`) ke liye yeh batata hai ke dictionary ka structure kya hoga.

**Key Point**: `TypedDict` ek actual class nahi hai jismein aap `__init__` banate hain ya object instantiate karte hain. Yeh ek type annotation hai jo runtime mein ek dictionary ke roop mein kaam karta hai.

---

### **2. `StopAtTools(stop_at_tool_names=["add"])` Kaise Kaam Karta Hai?**
Aapne `Agent` ko aise define kiya:

```python
agent = Agent(
    name="Math_Agent",
    instructions="You are Powerfull Assitant, Answer user math questions in better way PLease give improved anwser user question",
    tools=[add, fetch_weather],
    tool_use_behavior=StopAtTools(stop_at_tool_names=["add"])
)
```

Aapka sawal hai ke bina `StopAtTools` ka object banaye aur bina `__init__` ke, yeh direct call kaise kaam kar raha hai?

**Jawab**:
- **`TypedDict` Ka Behavior**: Python mein `TypedDict` classes ko directly dictionary ke taur par instantiate kiya ja sakta hai. Jab aap `StopAtTools(stop_at_tool_names=["add"])` likhte hain, Python isko ek dictionary banata hai:
  ```python
  {"stop_at_tool_names": ["add"]}
  ```
  Yeh dictionary `TypedDict` ke defined structure (`stop_at_tool_names: list[str]`) ko follow karta hai.

- **No `__init__` Needed**: `TypedDict` ke liye `__init__` constructor ki zarurat nahi hoti, kyunki yeh runtime mein ek regular dictionary hota hai. `TypedDict` ka asli kaam static type checking ke liye hota hai, taake tools jaise `mypy` check kar saken ke dictionary mein sahi keys aur types hain.

- **SDK Ka Role**: Jab aap `tool_use_behavior=StopAtTools(stop_at_tool_names=["add"])` pass karte hain, SDK is dictionary ko accept karta hai aur check karta hai ke yeh `StopAtTools` type ke mutabiq hai (yaani `stop_at_tool_names` key ke saath ek `list[str]`). SDK is dictionary ko use karta hai tool behavior ko control karne ke liye.

**Documentation Se Confirmation**:
- Documentation ke mutabiq, `tool_use_behavior` yeh types accept karta hai:
  ```python
  tool_use_behavior: Literal["run_llm_again", "stop_on_first_tool"] | StopAtTools | ToolsToFinalOutputFunction
  ```
  Yahaan `StopAtTools` ek `TypedDict` hai, jo ek dictionary ke roop mein pass kiya jata hai.

- Jab aap `StopAtTools(stop_at_tool_names=["add"])` likhte hain, yeh internally ek dictionary banata hai, jo SDK ke liye valid input hai.

---

### **3. Yeh Tareeka Galat Hai Kya?**
**Nahi, yeh tareeka bilkul galat nahi hai.** Yeh Python aur SDK ke design ke hisaab se bilkul sahi hai. Reasons:

1. **`TypedDict` Ka Syntax**: `TypedDict` ko directly aise call karna (`StopAtTools(key=value)`) Python mein ek standard practice hai. Yeh ek dictionary create karta hai jo type hints ke saath compatible hota hai.

2. **SDK Ka Design**: OpenAI Agents SDK expect karta hai ke `StopAtTools` ek dictionary ho jismein `stop_at_tool_names` key ho. Aapka code (`StopAtTools(stop_at_tool_names=["add"])`) yeh requirement fulfill karta hai.

3. **No Object Instantiation Needed**: `TypedDict` ke liye alag se object banane ki zarurat nahi, kyunki yeh runtime mein ek dictionary hota hai. `StopAtTools(stop_at_tool_names=["add"])` directly woh dictionary bana deta hai jo SDK ko chahiye.

---

### **4. Kaise Kaam Karta Hai?**
Jab aap yeh code chalate hain:

```python
agent = Agent(
    name="Math_Agent",
    instructions="...",
    tools=[add, fetch_weather],
    tool_use_behavior=StopAtTools(stop_at_tool_names=["add"])
)
result = Runner.run_sync(agent, "tell me whats the weather in karachi and What is 2 + 2 ?", run_config=config)
print(result.final_output)
```

- **Step-by-Step**:
  1. `StopAtTools(stop_at_tool_names=["add"])` ek dictionary banata hai: `{"stop_at_tool_names": ["add"]}`.
  2. Yeh dictionary `tool_use_behavior` ke taur par `Agent` ko pass hota hai.
  3. `Runner.run_sync` jab input ("What is 2 + 2 ?") process karta hai, toh agent `add` tool ko call karta hai, kyunki `stop_at_tool_names=["add"]` set hai.
  4. `add` tool ka output (`2 + 2 - 5 = -1`) final output ban jata hai, kyunki `StopAtTools` ke rule ke mutabiq agent ruk jata hai aur LLM result ko process nahi karta.

**Output**:
```
---------Tool Called Sucessfully-----------
-1  # result.final_output
```

---

### **5. Kyun Yeh Valid Hai?**
- `TypedDict` ko aise use karna (`StopAtTools(key=value)`) Python mein ek common aur valid tareeka hai.
- SDK is dictionary ko process karta hai aur `stop_at_tool_names` key se tool names nikal kar unke base par agent ka behavior control karta hai.
- `__init__` ya object instantiation ki zarurat nahi, kyunki `TypedDict` runtime mein dictionary ke roop mein kaam karta hai.

---

### **Summary**
- **Sahi Kaha Aapne**: Aapka code `tool_use_behavior=StopAtTools(stop_at_tool_names=["add"])` bilkul valid hai.
- **Kaise Kaam Karta Hai**: `StopAtTools` ek `TypedDict` hai, jo directly dictionary banata hai (`{"stop_at_tool_names": ["add"]}`) bina `__init__` ke. SDK is dictionary ko use karta hai tool behavior control karne ke liye.
- **Galat Nahi Hai**: Yeh tareeka bilkul sahi aur SDK ke design ke mutabiq hai. `TypedDict` ko aise hi use kiya jata hai jab type-safe dictionary chahiye.
---------------
---------------
----------------

Aapka sawal bilkul wajib hai! Aap pooch rahe hain ke jab `StopAtTools` ka maqsad sirf ek dictionary banana hai, toh iske liye `TypedDict` class ka use kyun kiya gaya jabki hum dictionary kisi bhi tareeke se bana sakte the (jaise direct `{"stop_at_tool_names": ["add"]}`)? Main isko aapke code aur OpenAI Agents SDK ke documentation ([https://openai.github.io/openai-agents-python/ref/agent/#agents.agent.Agent.tool_use_behavior](https://openai.github.io/openai-agents-python/ref/agent/#agents.agent.Agent.tool_use_behavior)) ke base par simple tareeke se samjhaunga.

---

### **Kyun `TypedDict` Class Use Kiya Gaya?**
`StopAtTools` ko `TypedDict` ke roop mein define karne ka maqsad **type safety** aur **code clarity** hai, na ke sirf dictionary banana. Aap sahi kehte hain ke dictionary direct bhi banaya ja sakta tha, lekin `TypedDict` use karne ke kuch specific fayde hain:

1. **Type Safety**:
   - `TypedDict` Python ke `typing` module ka hissa hai aur yeh batata hai ke dictionary ka structure kya hoga (jaise `stop_at_tool_names` ek `list[str]` hona chahiye).
   - Jab aap `StopAtTools(stop_at_tool_names=["add"])` likhte hain, static type checkers (jaise `mypy`) check karte hain ke dictionary mein sahi keys aur unke correct types hain.
   - Example: Agar aap galti se `stop_at_tool_names` ki jagah koi aur key ya galat type (jaise `int` ke bajaye `str`) dete hain, toh `mypy` error dikhayega:
     ```python
     StopAtTools(wrong_key=["add"])  # Type checker error dega
     StopAtTools(stop_at_tool_names=123)  # Type checker error dega
     ```

2. **Code Clarity**:
   - `TypedDict` se developers ko clear hota hai ke `tool_use_behavior` ke liye ek specific dictionary structure expect kiya jata hai. Yeh code ko self-documenting banata hai.
   - Bina `TypedDict` ke, agar aap direct dictionary dete (`{"stop_at_tool_names": ["add"]}`), toh doosre developers ko documentation ya code padhna padega yeh samajhne ke liye ke dictionary mein kaunse keys valid hain.

3. **SDK Ka Design**:
   - OpenAI Agents SDK ne `tool_use_behavior` ke liye ek union type define kiya hai:
     ```python
     tool_use_behavior: Literal["run_llm_again", "stop_on_first_tool"] | StopAtTools | ToolsToFinalOutputFunction
     ```
     Yahaan `StopAtTools` ek `TypedDict` hai, jo SDK ko yeh batata hai ke agar aap dictionary pass kar rahe hain, toh uska structure `StopAtTools` ke mutabiq hona chahiye.
   - Agar aap direct dictionary dete (`{"stop_at_tool_names": ["add"]}`), toh SDK shayad accept kar le, lekin type checkers error de sakte hain kyunki dictionary ka type explicitly `StopAtTools` ke saath match nahi hota.

4. **Maintainability**:
   - Agar future mein `StopAtTools` mein aur keys add karni hon (jaise `stop_at_tool_names` ke alawa koi aur field), toh `TypedDict` ka use karna asaan hai kyunki aap bas class definition update kar sakte hain:
     ```python
     class StopAtTools(TypedDict):
         stop_at_tool_names: list[str]
         new_field: str  # Future mein add ho sakta hai
     ```

---

### **Kya Direct Dictionary Use Kar Sakte Hain?**
Haan, technically aap direct dictionary (`{"stop_at_tool_names": ["add"]}`) `tool_use_behavior` mein pass kar sakte hain, kyunki runtime mein SDK isko accept kar lega (kyunki `TypedDict` runtime mein ek regular dictionary hota hai). Lekin yeh **recommended nahi hai** kyunki:

- **Type Safety Nahi**: Direct dictionary ke saath, type checkers (jaise `mypy`) yeh verify nahi kar payenge ke dictionary ka structure sahi hai ya nahi.
- **Code Less Clear**: Doosre developers ke liye yeh samajhna mushkil hoga ke dictionary mein kaunse keys valid hain bina documentation padhe.
- **Error Prone**: Agar aap galat key ya type dete hain, toh runtime errors aa sakte hain jo type checking ke waqt catch ho sakte the.

**Example (Direct Dictionary)**:
```python
agent = Agent(
    name="Math_Agent",
    instructions="...",
    tools=[add, fetch_weather],
    tool_use_behavior={"stop_at_tool_names": ["add"]}  # Direct dictionary
)
```

Yeh kaam toh kar sakta hai, lekin:
- `mypy` error de sakta hai kyunki `{"stop_at_tool_names": ["add"]}` ka type `StopAtTools` nahi hai.
- Agar aap galat key likh dete hain (jaise `stop_tool_names` galti se), toh SDK runtime mein error de sakta hai.

---

### **Kyun `TypedDict` Better Hai?**
Aapke code mein `StopAtTools(stop_at_tool_names=["add"])` ka use bilkul sahi hai kyunki:

1. **Type-Safe**: `StopAtTools` `TypedDict` ke zariye SDK aur type checkers ko pata hai ke dictionary mein `stop_at_tool_names` key aur `list[str]` type hona chahiye.
2. **Clear Intent**: Yeh code ko padhne wale ke liye clear karta hai ke `tool_use_behavior` ke liye yeh specific structure expect kiya jata hai.
3. **SDK Compatibility**: SDK `StopAtTools` type ke dictionary ko expect karta hai, aur yeh use karna SDK ke design ke mutabiq hai.

---

### **Kaise Kaam Karta Hai?**
Jab aap likhte hain:

```python
agent = Agent(
    name="Math_Agent",
    instructions="...",
    tools=[add, fetch_weather],
    tool_use_behavior=StopAtTools(stop_at_tool_names=["add"])
)
```

- `StopAtTools(stop_at_tool_names=["add"])` ek dictionary banata hai: `{"stop_at_tool_names": ["add"]}`.
- SDK is dictionary ko `tool_use_behavior` ke taur par accept karta hai aur check karta hai ke yeh `StopAtTools` type ke mutabiq hai.
- Jab `Runner.run_sync` chalata hai, SDK `stop_at_tool_names` key se tool names (jaise `"add"`) padhta hai aur agent ko us tool ke call hone par rok deta hai, jaisa documentation mein kaha gaya:
  > *"The agent will stop running if any of the tools in the list are called. The final output will be the output of the first matching tool call."*

---

### **Summary**
- **Kyun `TypedDict` Use Kiya?**:
  - `TypedDict` (`StopAtTools`) ka use type safety, code clarity, aur maintainability ke liye kiya gaya hai.
  - Yeh ensure karta hai ke dictionary ka structure sahi ho (jaise `stop_at_tool_names: list[str]`).
  - Direct dictionary bhi use ho sakta hai, lekin yeh type-safe nahi aur error-prone ho sakta hai.

- **Kya Yeh Galat Hai?**:
  - Nahi, `StopAtTools(stop_at_tool_names=["add"])` bilkul sahi tareeka hai aur SDK ke design ke mutabiq hai.
  - Direct dictionary use karna technically kaam kar sakta hai, lekin type checking aur clarity ke liye `TypedDict` better hai.

------------
------------
------------

# **4. ToolsToFinalOutputFunction in Tool_Use_Behaviour:**
---

### **1. `ToolsToFinalOutputFunction` Kya Hai?**
Documentation ke mutabiq, `ToolsToFinalOutputFunction` ek **TypeAlias** hai jo ek callable function ko represent karta hai. Yeh function `tool_use_behavior` ke liye ek option hai, aur iska kaam hai tool calls ke results ko process karke yeh decide karna ke agent ka final output kya hoga.

Type definition:
```python
ToolsToFinalOutputFunction: TypeAlias = Callable[
    [RunContextWrapper[TContext], list[FunctionToolResult]],
    MaybeAwaitable[ToolsToFinalOutputResult],
]
"""A function that takes a run context and a list of tool results, and returns a `ToolsToFinalOutputResult`."""
```

- **Simple Explanation**: Yeh ek custom function hai jo aap define karte hain. Jab agent koi tool(s) call karta hai, toh yeh function un tool results ko dekhta hai aur decide karta hai ke agent ka final output kya hoga aur kya agent ko rukna chahiye ya continue karna chahiye.

---

### **2. `tool_use_behavior` Mein Iska Kaam**
`tool_use_behavior` agent ke tools ke behavior ko control karta hai. Documentation ke mutabiq, `tool_use_behavior` ke chaar possible values hain:
1. `"run_llm_again"`: Tool call ke baad LLM result ko process karta hai.
2. `"stop_on_first_tool"`: Pehle tool ka output final output ban jata hai.
3. `StopAtTools`: Specific tool names ke call hone par agent rukta hai.
4. `ToolsToFinalOutputFunction`: Ek custom function jo tool results ko process karta hai aur final output decide karta hai.

**`ToolsToFinalOutputFunction` Ka Role**:
- Jab aap `tool_use_behavior` mein ek custom function pass karte hain (jo `ToolsToFinalOutputFunction` type ka hota hai), yeh function tool calls ke results ko lekar:
  - Decide karta hai ke tool ka output final output hoga ya nahi.
  - Yeh bhi batata hai ke agent ko aage chalna chahiye ya rukna chahiye.
- Yeh aapko **custom logic** dene ki flexibility deta hai, taake aap tool results ke base par apne hisaab se output aur agent ka behavior control kar saken.

**Documentation Se Quote**:
> *"A function: If you pass a function, it will be called with the run context and the list of tool results. It must return a `ToolToFinalOutputResult`, which determines whether the tool calls result in a final output."*

---

### **3. `ToolsToFinalOutputFunction` Mein Kaunse Parameters Aate Hain?**
`ToolsToFinalOutputFunction` ek callable function hai jo do parameters accept karta hai aur ek specific type ka output return karta hai. Yeh hain:

#### **Parameters**:
1. **`context: RunContextWrapper[TContext]`**:
   - Yeh `RunContextWrapper` type ka object hai, jismein aapka custom context (jaise `MyCustomContext`) wrapped hota hai.
   - `context.context` se aap apne context ke fields (jaise `name`, `age`) access kar sakte hain.
   - Iska kaam hai run ke context (jaise user ka data ya state) ko provide karna, taake aap isko apne logic mein use kar saken.

2. **`tool_results: list[FunctionToolResult]`**:
   - Yeh ek list hai jismein sare tool calls ke results hote hain.
   - Har `FunctionToolResult` ek tool call ka output hota hai (jaise aapke `add` function ka result `-1` jab input `2 + 2` ho).
   - Yeh parameter aapko sare executed tools ke outputs deta hai, taake aap inko process kar saken.

#### **Return Type**:
- **`MaybeAwaitable[ToolsToFinalOutputResult]`**:
  - Yeh function ek `ToolsToFinalOutputResult` object return karta hai, ya toh directly ya asynchronously (`await` ke saath).
  - `ToolsToFinalOutputResult` ek object hai jo batata hai:
    - **Kya tool ka output final output hai?** (Yani agent ko rukna chahiye.)
    - **Final output kya hai?** (Agar output final hai toh.)
  - Documentation mein `ToolsToFinalOutputResult` ka exact structure nahi diya gaya, lekin typically yeh ek dataclass ya dictionary hota hai jismein `is_final` (boolean) aur `output` (result) jaise fields hote hain.

**Example Signature**:
```python
def custom_tool_handler(context: RunContextWrapper[MyCustomContext], tool_results: list[FunctionToolResult]) -> ToolsToFinalOutputResult:
    # Custom logic
    pass
```

---

### **4. Yeh Kaise Kaam Karta Hai?**
Jab aap `tool_use_behavior` mein ek custom function pass karte hain, SDK yeh steps follow karta hai:

1. **Tool Call**: Agent koi tool(s) call karta hai (jaise `add` ya `fetch_weather`).
2. **Function Call**: SDK aapke custom function (`ToolsToFinalOutputFunction`) ko call karta hai, jismein:
   - `context`: Aapka `my_context` wrapped in `RunContextWrapper`.
   - `tool_results`: Sare tool calls ke results ki list.
3. **Output Decision**: Aapka function `ToolsToFinalOutputResult` return karta hai, jo SDK ko batata hai:
   - Kya yeh tool results final output hain?
   - Agar haan, toh final output kya hai?
4. **Agent Behavior**: SDK is result ke base par decide karta hai ke agent ko rukna hai ya continue karna hai.

**Example**:
Agar aap yeh function define karte hain:

```python
from agents.run import ToolsToFinalOutputResult

def custom_tool_handler(context: RunContextWrapper[MyCustomContext], tool_results: list[FunctionToolResult]) -> ToolsToFinalOutputResult:
    # Agar tool result mein "add" ka output hai, toh usko final output banao
    for result in tool_results:
        if result.tool_name == "add":
            return ToolsToFinalOutputResult(is_final=True, output=f"Result from add: {result.output}")
    # Warna continue karo
    return ToolsToFinalOutputResult(is_final=False, output=None)
```

Aur agent ko aise configure karte hain:

```python
agent = Agent(
    name="Math_Agent",
    instructions="...",
    tools=[add, fetch_weather],
    tool_use_behavior=custom_tool_handler  # Custom function
)
```

Jab aap `Runner.run_sync` chalate hain:

```python
result = Runner.run_sync(agent, "What is 2 + 2 ?", run_config=config)
print(result.final_output)
```

- Agar `add` tool call hota hai, toh `custom_tool_handler` `ToolsToFinalOutputResult(is_final=True, output="Result from add: -1")` return karega.
- Final output hoga: `"Result from add: -1"`.

---

### **5. Summary**
- **Kya Hai**: `ToolsToFinalOutputFunction` ek custom function hai jo tool results ko process karta hai aur final output decide karta hai.
- **Kaam**: Yeh `tool_use_behavior` mein use hota hai taake aap tool results ke base par custom logic se final output aur agent ka behavior control kar saken.
- **Parameters**:
  - `context: RunContextWrapper[TContext]`: Run ka context (jaise `my_context`).
  - `tool_results: list[FunctionToolResult]`: Tool calls ke results ki list.
- **Return**: `ToolsToFinalOutputResult`, jo batata hai ke output final hai ya nahi aur final output kya hai.
- **Kaise Kaam Karta Hai**: SDK is function ko tool calls ke baad call karta hai, aur yeh function decide karta hai ke agent ka final output kya hoga.

Agar aapko isko implement karne ke liye example ya aur clarification chahiye, toh batayein!

# **5. MaybeAwaitable :**
---

### **1. `MaybeAwaitable` Kya Hai?**
`MaybeAwaitable` ek **type hint** hai jo Python ke `typing` module ya SDK ke custom type system mein define hota hai. Iska matlab hai ke koi function ya toh ek value directly return kar sakta hai ya phir ek **awaitable** value (jaise coroutine ya future) return kar sakta hai, jisko `await` keyword ke saath resolve karna padta hai.

**Definition** (inferred from common Python async practices and SDK context):
```python
from typing import TypeVar, Awaitable, Union

T = TypeVar("T")
MaybeAwaitable: TypeAlias = Union[T, Awaitable[T]]
```

- **`T`**: Yeh generic type hai jo actual return value ka type hota hai (yahan `ToolsToFinalOutputResult`).
- **`Union[T, Awaitable[T]]`**: Iska matlab hai ke function ya toh directly `T` (yaani `ToolsToFinalOutputResult`) return karega, ya phir ek `Awaitable[T]` (jaise coroutine jo `await` hone par `ToolsToFinalOutputResult` deta hai).

**Simple Matlab**:
- `MaybeAwaitable[ToolsToFinalOutputResult]` ka matlab hai ke `ToolsToFinalOutputFunction` ya toh:
  - Direct `ToolsToFinalOutputResult` object return karega (synchronous function).
  - Ya phir ek coroutine return karega jo `await` hone par `ToolsToFinalOutputResult` dega (asynchronous function).

---

### **2. `ToolsToFinalOutputFunction` Mein `MaybeAwaitable` Ka Kaam**
`ToolsToFinalOutputFunction` ka type aisa hai:

```python
ToolsToFinalOutputFunction: TypeAlias = Callable[
    [RunContextWrapper[TContext], list[FunctionToolResult]],
    MaybeAwaitable[ToolsToFinalOutputResult],
]
```

- **Role of `MaybeAwaitable`**:
  - Yeh flexibility deta hai ke aapka custom function (`ToolsToFinalOutputFunction`) synchronous ya asynchronous dono ho sakta hai.
  - Yani, aap yeh function define kar sakte hain:
    - **Synchronous**: Direct `ToolsToFinalOutputResult` return kare.
    - **Asynchronous**: Ek coroutine return kare jo `await` hone par `ToolsToFinalOutputResult` de.

- **Kaam**:
  - SDK is function ko call karta hai jab tool results ko process karna hota hai (jaise jab `add` ya `fetch_weather` tool ka result aata hai).
  - Agar function synchronous hai, toh SDK directly uska return value (yaani `ToolsToFinalOutputResult`) leta hai.
  - Agar function asynchronous hai (coroutine return karta hai), toh SDK usko `await` karta hai taake `ToolsToFinalOutputResult` mile.

---

### **3. Parameters Aur Return**
**Parameters** (jo pehle samjha chuke hain):
- `context: RunContextWrapper[TContext]`: Run ka context (jaise `MyCustomContext`).
- `tool_results: list[FunctionToolResult]`: Tool calls ke results ki list.

**Return**:
- `MaybeAwaitable[ToolsToFinalOutputResult]`:
  - Yeh ek `ToolsToFinalOutputResult` object return karta hai, jo batata hai ke tool ka output final hai ya nahi, aur final output kya hai.
  - `MaybeAwaitable` isko flexible banata hai taake function synchronous ya asynchronous ho sake.

**Example**:
1. **Synchronous Function**:
```python
from agents.run import ToolsToFinalOutputResult

def custom_tool_handler(context: RunContextWrapper[MyCustomContext], tool_results: list[FunctionToolResult]) -> ToolsToFinalOutputResult:
    for result in tool_results:
        if result.tool_name == "add":
            return ToolsToFinalOutputResult(is_final=True, output=f"Result from add: {result.output}")
    return ToolsToFinalOutputResult(is_final=False, output=None)
```

2. **Asynchronous Function**:
```python
async def custom_tool_handler(context: RunContextWrapper[MyCustomContext], tool_results: list[FunctionToolResult]) -> ToolsToFinalOutputResult:
    # Maan lijiye kuch async operation (jaise API call)
    await asyncio.sleep(1)  # Example async operation
    for result in tool_results:
        if result.tool_name == "add":
            return ToolsToFinalOutputResult(is_final=True, output=f"Result from add: {result.output}")
    return ToolsToFinalOutputResult(is_final=False, output=None)
```

- **Synchronous Case**: `custom_tool_handler` directly `ToolsToFinalOutputResult` return karta hai.
- **Asynchronous Case**: `custom_tool_handler` ek coroutine return karta hai, jisko SDK `await` karta hai taake `ToolsToFinalOutputResult` mile.

---

### **4. Kaise Kaam Karta Hai?**
Jab aap `tool_use_behavior` mein `custom_tool_handler` set karte hain:

```python
agent = Agent(
    name="Math_Agent",
    instructions="...",
    tools=[add, fetch_weather],
    tool_use_behavior=custom_tool_handler
)
result = Runner.run_sync(agent, "What is 2 + 2 ?", run_config=config)
print(result.final_output)
```

- **SDK Ka Logic**:
  1. Agent `add` tool call karta hai.
  2. SDK `custom_tool_handler` ko call karta hai, aur:
     - Agar function synchronous hai, toh directly return value (`ToolsToFinalOutputResult`) leta hai.
     - Agar function asynchronous hai, toh SDK usko `await` karta hai taake `ToolsToFinalOutputResult` mile.
  3. `ToolsToFinalOutputResult` ke base par SDK decide karta hai ke final output kya hai aur agent ko rukna hai ya continue karna hai.

**Output Example** (agar `add` tool call hota hai):
```
---------Tool Called Sucessfully-----------
Result from add: -1  # result.final_output
```

---

### **5. Kyun `MaybeAwaitable` Use Kiya Gaya?**
- **Flexibility**: `MaybeAwaitable` SDK ko yeh flexibility deta hai ke developers synchronous ya asynchronous functions dono use kar saken.
- **Real-World Use Cases**: Kuch tools ya logic async operations involve karte hain (jaise API calls, database queries), toh asynchronous function support karna zaroori hai.
- **Type Safety**: `MaybeAwaitable` type hint ensure karta hai ke function ya toh direct `ToolsToFinalOutputResult` dega ya phir `await` hone par yeh dega.

---

### **Summary**
- **Kya Hai**: `MaybeAwaitable[ToolsToFinalOutputResult]` ek type hint hai jo kehta hai ke function ya toh directly `ToolsToFinalOutputResult` return karega ya phir ek coroutine jo `await` hone par `ToolsToFinalOutputResult` dega.
- **Kaam**: Yeh `ToolsToFinalOutputFunction` ko flexible banata hai taake aap synchronous ya asynchronous function use kar saken. SDK isko handle karta hai by checking if the return value needs to be awaited.
- **Parameters**: `context: RunContextWrapper[TContext]` aur `tool_results: list[FunctionToolResult]` function ke input hain, aur yeh `ToolsToFinalOutputResult` return karta hai.

Agar aapko isko implement karne ka example ya aur clarification chahiye, toh batayein!

--------------
# **6. Tool Choice in Model-Settting:**

### **Definition of `tool_choice`**
`tool_choice` ek setting hai jo control karta hai ke agent ka model (LLM) tools ko kaise use karega jab woh input process karta hai. Yeh batata hai ke model ko tools ka use karna chahiye ya nahi, aur agar karna hai toh kaise.

**Documentation se**:
```python
tool_choice: Literal["auto", "required", "none"] | str | None = None
"""The tool choice to use when calling the model."""
```

**Simple Matlab**:
- `tool_choice` decide karta hai ke model (jaise GPT-4o) ko tools call karne ka decision kaise lena hai.
- Yeh ek optional parameter hai (default `None`), jo model ke behavior ko customize karta hai jab tools available hote hain.

---

### **Teeno Parameters (Values) aur Unka Matlab**
`tool_choice` ke teeno main values hain: `"auto"`, `"required"`, aur `"none"`. Inko simple tareeke se samjhte hain:

1. **`"auto"`**:
   - **Matlab**: Model khud decide karta hai ke tool call karna hai ya nahi, input ke hisaab se.
   - **Kaam**: Agar input mein tool use karna zaroori lagta hai (jaise "2 + 2" ke liye `add` tool), toh model tool call karega. Agar nahi, toh direct jawab dega.
   - **Example**: Agar input hai "What is 2 + 2?", model `add` tool call kar sakta hai. Agar input hai "Hello", toh shayad tool na call kare.

2. **`"required"`**:
   - **Matlab**: Model ko har haal mein ek tool call karna hi hoga, chahe input kuch bhi ho.
   - **Kaam**: Model force hota hai ke available tools mein se koi ek tool zaroor use kare, bhale hi input direct jawab mangta ho.
   - **Example**: Agar input hai "What is 2 + 2?", model `add` tool call karega. Agar input hai "Hello", toh bhi model koi tool (jaise `fetch_weather`) call karne ki koshish karega.

3. **`"none"`**:
   - **Matlab**: Model koi bhi tool call nahi karega, chahe tools available hon.
   - **Kaam**: Model sirf apne internal knowledge ya instructions ke base par jawab dega, tools ko ignore karke.
   - **Example**: Agar input hai "What is 2 + 2?", model `add` tool use nahi karega aur shayad direct jawab dega (jaise "4").

**Extra Values**:
- **`str`**: Aap ek specific tool ka naam de sakte hain (jaise `"add"`), toh model sirf usi tool ko call karega jab input ke liye suitable ho.
- **`None` (default)**: Yeh typically `"auto"` jaisa behave karta hai, lekin model ke default settings par depend karta hai.

---

### **Kaise Kaam Karta Hai?**
- Jab aap `Agent` define karte hain aur `tool_choice` set karte hain, SDK yeh batata hai ke model ko tools ka use kaise karna hai.
- Example:
  ```python
  agent = Agent(
      name="Math_Agent",
      instructions="Answer math questions using tools.",
      tools=[add, fetch_weather],
      tool_choice="required"  # Force tool use
  )
  result = Runner.run_sync(agent, "What is 2 + 2?", run_config=config)
  print(result.final_output)
  ```
  - Agar `tool_choice="required"`, model `add` tool ko call karega, kyunki input math-related hai.
  - Agar `tool_choice="none"`, model tool use nahi karega aur shayad direct "4" jawab dega.

---

### **Summary**
- **Definition**: `tool_choice` control karta hai ke model tools ko kaise use karega.
- **Teeno Values**:
  - `"auto"`: Model khud decide karta hai ke tool call karna hai ya nahi.
  - `"required"`: Model ko har haal mein tool call karna hota hai.
  - `"none"`: Model koi tool call nahi karta, sirf apne knowledge se jawab deta hai.
- **Aur Options**: `str` (specific tool name) ya `None` (default, usually `"auto"` jaisa).

Agar aapko isko test karne ke liye example ya aur clarification chahiye, toh batayein!

# **7. reset_tool_choice in Agent Class:**

### **Definition of `reset_tool_choice`**
`reset_tool_choice` ek boolean setting hai jo control karta hai ke ek tool call hone ke baad agent ka `tool_choice` setting wapas default value par reset ho ya nahi.

**Documentation se**:
```python
reset_tool_choice: bool = True
"""Whether to reset the tool choice to the default value after a tool has been called. Defaults to True. This ensures that the agent doesn't enter an infinite loop of tool usage."""
```

**Simple Matlab**:
- Yeh setting decide karta hai ke jab agent koi tool call karta hai (jaise `add` ya `fetch_weather`), toh kya uske `tool_choice` setting ko wapas default value (typically `"auto"`) par set karna hai ya nahi.
- Default value `True` hai, yani tool call hone ke baad `tool_choice` reset ho jata hai, taake agent bar-bar ek hi tool ko call na kare aur infinite loop mein na phase.

---

### **Iska Kaam Kya Hai?**
`reset_tool_choice` ka main kaam hai agent ke tool usage ko control karna aur infinite loops se bachana. Jab agent ek tool call karta hai, toh yeh setting ensure karta hai ke agent dobara usi tool ko bar-bar na chalaaye, balki naye input ke hisaab se naye decisions le.

**Simple Explanation**:
- **Jab `reset_tool_choice=True` (default)**:
  - Tool call hone ke baad, agent ka `tool_choice` wapas default value (usually `"auto"`) par reset ho jata hai.
  - Iska matlab hai ke agle turn mein agent input ke hisaab se khud decide karega ke koi tool call karna hai ya nahi, aur purana `tool_choice` setting (jaise `"required"` ya specific tool name) clear ho jata hai.
  - Yeh infinite loops se bachata hai, jaise agar agent bar-bar ek hi tool ko call karta rahe.

- **Jab `reset_tool_choice=False`**:
  - Tool call hone ke baad bhi `tool_choice` setting wahi rehta hai jo aapne set kiya tha (jaise `"required"`, `"none"`, ya specific tool name).
  - Is case mein, agent agle turn mein bhi wahi `tool_choice` setting follow karega, jo shayad bar-bar tool calls ya unexpected behavior ka karan ban sakta hai.

**Documentation ka Point**:
- Documentation kehti hai ke `reset_tool_choice=True` "ensures that the agent doesn't enter an infinite loop of tool usage." Yani, yeh setting agent ko stuck hone se rokti hai.

---

### **Example Se Samjhein**
Aapka code kuch aisa hai:

```python
agent = Agent(
    name="Math_Agent",
    instructions="Answer math questions using tools.",
    tools=[add, fetch_weather],
    tool_choice="required",  # Har turn mein tool call karna zaroori
    reset_tool_choice=True   # Default, tool call ke baad reset
)
result = Runner.run_sync(agent, "What is 2 + 2?", run_config=config)
print(result.final_output)
```

**Kaise Kaam Karta Hai?**:
1. **Pehla Turn**:
   - Input: "What is 2 + 2?"
   - `tool_choice="required"`, toh agent `add` tool call karta hai (kyunki yeh math-related hai).
   - `add(2, 2)` ka result `-1` aata hai.
   - `reset_tool_choice=True` hone ke wajah se, tool call ke baad `tool_choice` wapas default value (`"auto"`) par reset ho jata hai.

2. **Agla Turn** (agar hota):
   - Agar input aata hai (jaise multi-turn scenario mein), toh ab `tool_choice` `"auto"` hoga.
   - Agent input ke hisaab se decide karega ke tool call karna hai ya nahi, na ke force karega (`"required"` ke jaisa).
   - Yeh ensure karta hai ke agent bar-bar `add` tool na call kare agar input ke liye tool zaroori na ho.

**Agar `reset_tool_choice=False` Hota**:
- Tool call ke baad bhi `tool_choice` `"required"` rehta.
- Agle turn mein bhi agent kisi na kisi tool ko call karne ki koshish karega, chahe input tool-related na ho. Yeh shayad unwanted tool calls ya loop ka karan ban sakta hai.

---

### **Parameters**
`reset_tool_choice` ka sirf ek parameter hai:
- **Type**: `bool` (yaani `True` ya `False`).
- **Default**: `True` (yani tool call ke baad `tool_choice` reset hota hai).
- **Kaam**: Yeh batata hai ke tool call ke baad `tool_choice` ko default value (`"auto"`) par reset karna hai ya nahi.

---

### **Summary**
- **Kya Hai**: `reset_tool_choice` ek boolean setting hai jo control karta hai ke tool call ke baad `tool_choice` reset ho ya nahi.
- **Kaam**:
  - `True` (default): Tool call ke baad `tool_choice` `"auto"` par reset ho jata hai, taake agent infinite loop mein na phase.
  - `False`: `tool_choice` wahi rehta hai, jo shayad bar-bar tool calls ka karan ban sakta hai.
- **Parameter**: Sirf ek boolean value (`True` ya `False`), default `True`.

-------------

# **8. Tool Choice"required" - Infinite Loop behind the scene:**

### **Aapka Statement**
Aapne kaha:

"Yani OpenAI-Agents SDK mein Agent loop ke behind the aise hota hai ke Agent loop hamari input, system prompt aur available tools ko llm ke pass le kar jata hai through api jis mein model settings bhi hoti hai jis ke ander tool choice required hai toh phir llm input ki base par decide karta hai ke tool call karo phir agent loop tool call karta hai aur us ke output ki base par phir llm ko request karta hai jis mein tool output ke sath input aur system prompt aur model setting required llm ke pass dobara jati hai phir llm dekhta hai ke required hai tool choice toh phir wo tool call karta hai jab ke output beshak a gia ho isi tarah ye infinite loop mein chalta rehta hai."

---

### **Kya Aapne Sahi Kaha?**
Main aapke statement ko step-by-step check karta hoon:

1. **"Agent loop hamari input, system prompt aur available tools ko llm ke pass le kar jata hai through api jis mein model settings bhi hoti hai jis ke ander tool choice required hai"**
   - **Sahi Hai**: Agent loop yahi kaam karta hai. Woh user input, system prompt, aur available tools ko LLM ke paas API ke through bhejta hai. Model settings mein `tool_choice="required"` bhi include hota hai, jo LLM ko batata hai ke har baar tool call karna zaroori hai.

2. **"Toh phir llm input ki base par decide karta hai ke tool call karo"**
   - **Sahi Hai**: LLM input ko dekhta hai aur decide karta hai ke kaunsa tool call karna hai. Lekin jab `tool_choice="required"` hota hai, toh woh tool call zaroor karega, chahe input ke hisaab se zarurat ho ya na ho.

3. **"Phir agent loop tool call karta hai aur us ke output ki base par phir llm ko request karta hai jis mein tool output ke sath input aur system prompt aur model setting required llm ke pass dobara jati hai"**
   - **Sahi Hai**: Agent loop tool ko call karta hai, aur tool ka output milne ke baad, woh output ke saath input, system prompt, aur model settings (jismein `tool_choice="required"` ab bhi hai) dobara LLM ko bhejta hai.

4. **"Phir llm dekhta hai ke required hai tool choice toh phir wo tool call karta hai jab ke output beshak a gia ho"**
   - **Sahi Hai**: LLM dobara request ko dekhta hai, aur kyunki `tool_choice="required"` ab bhi set hai, woh phir se tool call karta hai, chahe pehle tool ka output aa chuka ho. Yeh behavior tab tak chalta hai jab tak settings change na ho ya loop rokne ka koi limit na lage.

5. **"Isi tarah ye infinite loop mein chalta rehta hai"**
   - **Sahi Hai**: Jab tak `tool_choice="required"` set hai aur har baar tool call ke baad LLM ko wapas request jati hai, yeh process bar-bar chalta rahega. Yeh infinite loop tab ban sakta hai jab agent final output produce na kare ya loop ko rokne ka koi mechanism (jaise `max_turns`) na ho.

---

### **Mera Jawab**
Haan, aapne bilkul sahi kaha hai. OpenAI Agents SDK mein agent loop ke behind the scene yahi hota hai ke:

- Agent loop hamari input, system prompt, aur available tools ko LLM ke paas le kar jata hai through API, jismein model settings hoti hain aur `tool_choice="required"` set hota hai.
- Phir LLM input ki base par decide karta hai ke tool call karo, aur kyunki "required" hai, toh woh tool call karta hai.
- Agent loop tool call karta hai, aur uske output ke baad phir LLM ko request bhejta hai jismein tool output ke saath input, system prompt, aur model settings (with `tool_choice="required"`) dobara jati hain.
- LLM dekhta hai ke `tool_choice="required"` hai, toh woh phir tool call karta hai, chahe output aa chuka ho.
- Isi tarah yeh infinite loop mein chalta rehta hai, jab tak koi limit ya condition loop ko na roke.

-------------------------


# **9. asyncio.get_event_loop().run_until_complete methods in Run_Sync:**

Aapka sawal yeh hai ke aapko samajhna hai ke is code snippet mein kya ho raha hai aur yeh kaise kaam kar raha hai, jabke `run` method asynchronous hai lekin aapne `run_sync` ko call kiya hai jo synchronous hai. Chaliye isko step-by-step samajhte hain taake aapko bilkul clear ho jaye.

Yeh hai aapka code snippet:

```python
return asyncio.get_event_loop().run_until_complete(
    cls.run(
        starting_agent,
        input,
        context=context,
        max_turns=max_turns,
        hooks=hooks,
        run_config=run_config,
        previous_response_id=previous_response_id,
    )
)
```

---

### **Code Mein Kya Ho Raha Hai?**

Is code ka maqsad yeh hai ke ek asynchronous method (`cls.run`) ko synchronous tareeke se chalaya jaye. Yeh Python ke `asyncio` library ka use karta hai. Aaiye isko tod kar dekhte hain:

1. **`asyncio.get_event_loop()`**:
   - Yeh current thread ka event loop retrieve karta hai. Event loop asynchronous programming ka core hota hai jo async tasks ko manage karta hai.
   - Agar koi event loop nahi hai, toh yeh ek naya event loop create karta hai.

2. **`run_until_complete(...)`**:
   - Yeh method ek coroutine (asynchronous function) ko leta hai aur usko tab tak chalata hai jab tak woh complete na ho jaye.
   - Yeh current thread ko block karta hai, yani yeh wait karta hai jab tak coroutine ka kaam poora na ho jaye. Is wajah se yeh synchronous behavior deta hai.

3. **`cls.run(...)`**:
   - Yeh ek asynchronous method hai jo class (`cls`) ka part hai. Yeh shayad agent ke operations ko handle karta hai, jaise input process karna ya tools ko call karna.
   - Is method ko kai parameters pass kiye gaye hain:
     - `starting_agent`: Shuruaati agent.
     - `input`: Input data.
     - `context`: Additional context.
     - `max_turns`: Maximum turns ya iterations.
     - `hooks`: Lifecycle events ke liye hooks.
     - `run_config`: Run ka configuration.
     - `previous_response_id`: Pehle ke response ka ID.

---

### **`run` Asynchronous Hai, Phir `run_sync` Synchronous Kaise?**

Aapne kaha ke aapne `run_sync` ko call kiya hai, jo synchronous hai, lekin `run` method asynchronous hai. Yeh samajhne ke liye yeh jaan'na zaroori hai ke `run_sync` ek wrapper method hai jo asynchronous `run` ko synchronous tareeke se chalane ke liye banaya gaya hai.

**`run_sync` Ka Possible Implementation**:
Aapka code snippet yeh dikhata hai ke `run_sync` shayad aise implement kiya gaya hai:

```python
@classmethod
def run_sync(cls, starting_agent, input, context=None, max_turns=10, hooks=None, run_config=None, previous_response_id=None):
    return asyncio.get_event_loop().run_until_complete(
        cls.run(
            starting_agent,
            input,
            context=context,
            max_turns=max_turns,
            hooks=hooks,
            run_config=run_config,
            previous_response_id=previous_response_id,
        )
    )
```

- **Kaise Kaam Karta Hai**:
  - Jab aap `run_sync` ko call karte hain, yeh internally `run` method ko call karta hai, jo ek coroutine return karta hai.
  - Phir `run_until_complete` us coroutine ko event loop mein chalata hai aur wait karta hai jab tak woh complete na ho jaye.
  - Yeh blocking call hoti hai, yani `run_sync` tab tak wapas nahi aata jab tak `run` ka kaam poora na ho jaye. Isliye yeh synchronous ban jata hai.

---

### **Simple Example Se Samajhiye**

Maaniye ke `run` method ek asynchronous operation hai jo 5 seconds tak chalta hai (jaise ek API call). Agar aap seedha `run` ko call karenge, toh aapko `await` karna padega, jaise:

```python
result = await cls.run(starting_agent, input)
```

Lekin jab aap `run_sync` call karte hain:

```python
result = cls.run_sync(starting_agent, input)
```

- Yeh 5 seconds tak block karega (wait karega) aur phir result return karega.
- Internally, yeh `run` ko async tareeke se chalata hai lekin aapko synchronous experience deta hai.

---

### **Aapke Sawal Ka Jawab**

- **Kya Ho Raha Hai?**  
  Aapka code snippet dikhata hai ke `run_sync` method asynchronous `run` method ko `asyncio.get_event_loop().run_until_complete` ke zariye chalata hai. Yeh event loop start karta hai, `run` ko complete karta hai, aur result return karta hai.

- **`run` Async Hai, Phir Bhi Sync Kaise Chala?**  
  `run` method khud asynchronous hai, lekin `run_sync` usko synchronous banata hai kyunki woh `run_until_complete` ka use karta hai. Jab aap `run_sync` call karte hain, yeh wait karta hai jab tak `run` complete na ho jaye, toh aapko lagta hai ke yeh synchronous hai.

---

### **Summary**

- **`run` Method**: Yeh asynchronous hai, background mein operations chalata hai.
- **`run_sync` Method**: Yeh `run` ka synchronous wrapper hai jo event loop ke zariye `run` ko chalata hai aur complete hone ka wait karta hai.
- **Aapka Code**: Jab aap `run_sync` ko call karte hain, yeh internally `run` ko asynchronous tareeke se chala raha hai lekin aapko synchronous result deta hai kyunki `run_until_complete` block karta hai.
-----------------------

# **10. Custom Function Tool Class Behind the scene:**

Main aapke sawalon ka jawab step-by-step deta hoon taake aapko code aur uski working bilkul clear ho jaye. Aapne poocha hai ke `run_function` ke andar `args` kahan se aata hai, kaun isko pass karta hai, iski backend working kaise hoti hai, aur do methods `.model_json_schema()` aur `.model_validate_json()` kya hain aur kya kaam karte hain. Chaliye shuru karte hain!

---

### **1. `args` Kahan Se Aata Hai Aur Kaun Pass Karta Hai?**

`run_function` ka signature yeh hai:
```python
async def run_function(ctx: RunContextWrapper[Any], args: str) -> str:
    parsed = FunctionArgs.model_validate_json(args)
    return do_some_work(data=f"{parsed.username} is {parsed.age} years old")
```
- **`args` Kya Hai?**  
  `args` ek string parameter hai jo JSON format mein data ko hold karta hai, jaise `{"username": "Ali", "age": 25}`.

- **`args` Kahan Se Aata Hai?**  
  `args` ko **OpenAI Agents SDK** pass karta hai jab agent `FunctionTool` (jo `tool` variable mein define kiya gaya hai) ko call karta hai. Yeh tool ka naam hai `"process_user"`.

- **Kaun Pass Karta Hai?**  
  Jab agent ko `"process_user"` tool use karna hota hai, OpenAI Agents SDK `run_function` ko invoke karta hai aur ismein `ctx` (context) aur `args` (input data) pass karta hai. Yeh `args` agent ke internal logic ya user input se generate hota hai.

- **Example**  
  Maan lijiye agent yeh input deta hai:  
  ```json
  {"username": "Ali", "age": 25}
  ```  
  Yeh JSON string `args` ke roop mein `run_function` ko milti hai.

---

### **2. Backend Working Kaise Hoti Hai?**

Ab yeh samajhte hain ke yeh code backend mein kaise kaam karta hai:

1. **Agent Tool Ko Call Karta Hai**  
   - Code mein `tool = FunctionTool(...)` ek tool define karta hai jiska naam `"process_user"` hai.
   - Jab agent ko yeh tool chahiye hota hai, woh OpenAI Agents SDK ke zariye `on_invoke_tool` (jo `run_function` hai) ko call karta hai aur `args` provide karta hai.

2. **`run_function` Ka Logic**  
   - `run_function` ko `args` milta hai, jo ek JSON string hai (jaise `{"username": "Ali", "age": 25}`).
   - Phir `FunctionArgs.model_validate_json(args)` call hota hai, jo is JSON string ko parse karke `FunctionArgs` object mein convert karta hai. Is object ko `parsed` variable mein store kiya jata hai.
   - `parsed` mein `username` aur `age` fields hote hain, jaise `parsed.username = "Ali"` aur `parsed.age = 25`.
   - Phir yeh data `do_some_work` function ko diya jata hai ek string ke roop mein: `"Ali is 25 years old"`.
   - `do_some_work` function bas `"done"` return karta hai.

3. **Output**  
   - `"done"` string `run_function` se return hoti hai aur agent ko wapas milti hai. Agent is output ko apne logic ke hisaab se use karta hai.

**Flow Summary**:  
Agent → Tool Call (`process_user`) → `run_function` → JSON Parse → Data Process → Output (`"done"`) → Agent.

---

### **3. `.model_json_schema()` Kya Hai Aur Kya Kaam Karta Hai?**

- **Kya Hai?**  
  Yeh Pydantic model (`FunctionArgs`) ka ek method hai jo us model ka JSON schema generate karta hai. JSON schema ek structure hota hai jo batata hai ke data kaisa hona chahiye.

- **Code Mein Istemal**  
  ```python
  params_json_schema=FunctionArgs.model_json_schema()
  ```  
  Yeh `FunctionTool` ke `params_json_schema` argument mein use hota hai.

- **Kaam Kya Hai?**  
  - Yeh ek dictionary return karta hai jo `FunctionArgs` ke fields (`username` aur `age`) ka schema define karta hai.
  - Example Output:  
    ```json
    {
        "type": "object",
        "properties": {
            "username": {"type": "string"},
            "age": {"type": "integer"}
        },
        "required": ["username", "age"]
    }
    ```
  - Yeh schema agent ko batata hai ke `"process_user"` tool ko ek object chahiye jismein `username` (string) aur `age` (integer) hona zaroori hai.

- **Purpose**  
  Agent is schema ke zariye samajhta hai ke tool ko kaunsa input dena hai, taake woh sahi format mein `args` pass kar sake.

---

### **4. `.model_validate_json()` Kya Hai Aur Kya Kaam Karta Hai?**

- **Kya Hai?**  
  Yeh bhi Pydantic model (`FunctionArgs`) ka ek method hai jo JSON string ko parse karta hai, validate karta hai, aur usko `FunctionArgs` object mein convert karta hai.

- **Code Mein Istemal**  
  ```python
  parsed = FunctionArgs.model_validate_json(args)
  ```  
  Yeh `run_function` mein `args` ko process karne ke liye use hota hai.

- **Kaam Kya Hai?**  
  - `args` (jo ek JSON string hai) ko leta hai.
  - Check karta hai ke yeh string `FunctionArgs` ke schema ke mutabiq hai ya nahi (yani `username` string ho, `age` integer ho).
  - Agar data sahi hai, toh ek `FunctionArgs` object banata hai (`parsed`) jismein `parsed.username` aur `parsed.age` accessible hote hain.
  - Agar data galat hai, toh validation error throw karta hai.

- **Example**  
  - Agar `args = '{"username": "Ali", "age": 25}'`:  
    → `parsed = FunctionArgs(username="Ali", age=25)` banega.  
  - Agar `args = '{"username": "Ali", "age": "twenty"}'`:  
    → Error aayega kyunki `age` integer nahi hai.

- **Purpose**  
  Yeh ensure karta hai ke input data sahi format mein ho aur aasani se use kiya ja sake.

---

### **Summary**

- **`args` Kahan Se Aata Hai?**  
  `args` OpenAI Agents SDK se aata hai jab agent `"process_user"` tool ko call karta hai. Yeh JSON string mein input data hota hai.

- **Backend Working**  
  Agent tool ko call karta hai, `run_function` `args` ko parse aur process karta hai, aur result agent ko wapas deta hai.

- **`.model_json_schema()`**  
  `FunctionArgs` ka schema banata hai jo agent ko tool ke input format batata hai.

- **`.model_validate_json()`**  
  JSON string ko parse aur validate karke `FunctionArgs` object mein convert karta hai.

------------------

## LLM ko kaise pta chala ke fucntion ke args mein kia data ae ga:

Aapka sawal bilkul clear hai. Aap pooch rahe hain ke agent ko kaise pata chalta hai ke `run_function` ke `args` parameter mein kaunsa data aayega, aur yeh kaise `FunctionArgs` class ke JSON schema se related hai jo humne banaya tha. Main isko aapke code aur OpenAI Agents SDK ke documentation ([https://openai.github.io/openai-agents-python/tools/#custom-function-tools](https://openai.github.io/openai-agents-python/tools/#custom-function-tools)) ke base par simple aur focused tareeke se samjhaunga.

---

### **Code Recap**
Aapka code yeh hai:

```python
from typing import Any
from pydantic import BaseModel
from agents import RunContextWrapper, FunctionTool

def do_some_work(data: str) -> str:
    return "done"

class FunctionArgs(BaseModel):
    username: str
    age: int

async def run_function(ctx: RunContextWrapper[Any], args: str) -> str:
    parsed = FunctionArgs.model_validate_json(args)
    return do_some_work(data=f"{parsed.username} is {parsed.age} years old")

tool = FunctionTool(
    name="process_user",
    description="Processes extracted user data",
    params_json_schema=FunctionArgs.model_json_schema(),
    on_invoke_tool=run_function,
)
```

Aapka sawal hai: **Agent ko kaise pata chalta hai ke `args` mein kaunsa data aayega, aur yeh `FunctionArgs` ke JSON schema se kaise connected hai?**

---

### **Agent Ko Kaise Pata Chalta Hai?**

Agent ko yeh pata chalta hai ke `args` mein kaunsa data aayega **JSON schema** ke zariye, jo aapne `FunctionArgs.model_json_schema()` se provide kiya hai. Yeh schema `FunctionTool` ke `params_json_schema` parameter mein pass hota hai. Chaliye step-by-step dekhte hain:

1. **JSON Schema Ka Role**:
   - `FunctionArgs.model_json_schema()` ek JSON schema generate karta hai jo batata hai ke `process_user` tool ko kaunsa input data chahiye. Yeh schema aisa dikhta hai:
     ```json
     {
         "type": "object",
         "properties": {
             "username": {"type": "string"},
             "age": {"type": "integer"}
         },
         "required": ["username", "age"]
     }
     ```
   - Yeh schema agent ko batata hai ke `process_user` tool ke liye input ek JSON object hona chahiye jismein `username` (string) aur `age` (integer) dono zaroori hain.

2. **Agent Ka Decision-Making**:
   - Jab agent ko koi input milta hai (jaise user ka query: "Process user data for Ali, age 25"), OpenAI Agents SDK ka internal logic (jo LLM ke saath kaam karta hai) input aur tool ke schema ko dekhta hai.
   - SDK check karta hai ke kaunsa tool input ke liye suitable hai. `params_json_schema` ke zariye agent samajhta hai ke `process_user` tool ke liye `username` aur `age` wala data chahiye.
   - LLM input ko analyze karta hai aur schema ke mutabiq ek JSON string banata hai, jaise `{"username": "Ali", "age": 25}`.

3. **Args Kaise Pass Hota Hai?**:
   - Jab agent `process_user` tool ko call karta hai, SDK `run_function` ko invoke karta hai aur `args` parameter mein yeh JSON string pass karta hai.
   - Yeh JSON string (`args`) wohi data hota hai jo `FunctionArgs` schema ke mutabiq hai, yani `username` aur `age` wala object.

4. **Validation in `run_function`**:
   - `run_function` ke andar, `FunctionArgs.model_validate_json(args)` yeh JSON string ko parse karta hai aur check karta hai ke yeh schema ke mutabiq hai ya nahi.
   - Agar data sahi hai (jaise `{"username": "Ali", "age": 25}`), toh yeh `FunctionArgs` object banata hai. Agar galat hai (jaise `age` string ho), toh error throw karta hai.

---

### **Backend Working Kaise Hoti Hai?**

Yeh process step-by-step aise kaam karta hai:

1. **Agent Input Ko Process Karta Hai**:
   - User input (jaise "Process user data for Ali, age 25") agent ko milta hai.
   - Agent ke instructions aur `tool_choice` setting ke base par, SDK decide karta hai ke kaunsa tool call karna hai. Is case mein, `process_user` tool suitable hai kyunki input user data se related hai.

2. **Schema Se Match Karta Hai**:
   - `FunctionTool` ke `params_json_schema` mein diya gaya JSON schema agent ko batata hai ke `process_user` tool ko `username` aur `age` chahiye.
   - LLM input ko dekhta hai aur schema ke mutabiq ek JSON string banata hai, jaise `{"username": "Ali", "age": 25}`.

3. **Tool Invocation**:
   - Jab agent `process_user` tool ko call karta hai, SDK `on_invoke_tool` (yaani `run_function`) ko call karta hai aur `args` mein yeh JSON string pass karta hai.

4. **Run Function Mein Processing**:
   - `run_function` `args` ko `FunctionArgs.model_validate_json(args)` ke zariye parse karta hai.
   - Yeh `FunctionArgs` object banata hai jismein `username` aur `age` accessible hote hain.
   - Phir yeh data `do_some_work` ko pass karta hai aur result (`"done"`) return karta hai.

**Flow Summary**:
- User Input → Agent (LLM) → Schema Match → JSON String (`args`) → `run_function` → Parse (`model_validate_json`) → Process (`do_some_work`) → Output.

---

### **`.model_json_schema` aur `.model_validate_json` Ka Role**

Yeh dono methods aapne pehle sawal mein bhi poocha tha, lekin yahan inka connection `args` ke saath samjha deta hoon:

1. **`.model_json_schema()`**:
   - **Kya Hai**: Yeh Pydantic model (`FunctionArgs`) ka JSON schema generate karta hai.
   - **Kaam**: Yeh agent ko batata hai ke `process_user` tool ko kaunsa data chahiye (yani `username: str`, `age: int`).
   - **Connection with `args`**: Yeh schema agent ko guide karta hai ke input ko kaise JSON string mein convert karna hai taake woh `args` ke roop mein `run_function` ko pass ho sake.
   - **Example**: Schema ke base par, agent `{"username": "Ali", "age": 25}` banata hai aur yeh `args` ke roop mein pass hota hai.

2. **`.model_validate_json()`**:
   - **Kya Hai**: Yeh JSON string (`args`) ko parse karta hai aur `FunctionArgs` object mein convert karta hai, sath hi validate karta hai ke data schema ke mutabiq hai.
   - **Kaam**: Yeh ensure karta hai ke `args` mein diya gaya data sahi format mein hai (yani `username` string aur `age` integer).
   - **Connection with `args`**: `run_function` mein yeh method `args` ko process karta hai aur `parsed` object banata hai, jisse `username` aur `age` access ho sakein.
   - **Example**: `args = '{"username": "Ali", "age": 25}'` → `parsed = FunctionArgs(username="Ali", age=25)`.

---

### **Agent Ko Schema Se Kaise Pata Chalta Hai?**

- **`params_json_schema`**: `FunctionTool` ke `params_json_schema` mein `FunctionArgs.model_json_schema()` se bana schema agent ko milta hai jab tool initialize hota hai.
- **LLM Ka Role**: Agent ka LLM (jaise GPT-4o) input ko dekhta hai aur schema ke base par decide karta hai ke `process_user` tool suitable hai ya nahi. Woh input se relevant data (jaise username aur age) nikaalta hai aur schema ke mutabiq JSON string banata hai.
- **Data Passing**: Yeh JSON string (`args`) `run_function` ko pass hoti hai, aur `model_validate_json` isko validate aur process karta hai.

**Example**:
- Input: "Process user data for Ali, age 25"
- Schema: `{"username": {"type": "string"}, "age": {"type": "integer"}}`
- Agent banata hai: `args = '{"username": "Ali", "age": 25}'`
- `run_function` mein: `parsed = FunctionArgs.model_validate_json(args)` → `parsed.username = "Ali"`, `parsed.age = 25`

---

### **Summary**

- **Agent Ko Kaise Pata Chalta Hai?**: Agent ko `FunctionArgs.model_json_schema()` se bana JSON schema batata hai ke `args` mein kaunsa data aayega (yani `username` aur `age` wala JSON object).
- **Kaun Pass Karta Hai?**: OpenAI Agents SDK aur LLM input ke base par schema ke mutabiq JSON string (`args`) banate hain aur `run_function` ko pass karte hain.
- **Backend Working**: Agent input ko schema ke saath match karta hai, JSON string banata hai, aur `run_function` mein validate aur process karta hai.
- **`.model_json_schema`**: Tool ke input ka structure define karta hai taake agent sahi data bheje.
- **`.model_validate_json`**: `args` JSON string ko parse aur validate karke `FunctionArgs` object banata hai.

--------------
--------------
--------------

# **11. RunContextWrapper and args:**

Aapka sawal bilkul valid hai. Aap pooch rahe hain ke jab hum `FunctionArgs` ka JSON schema define karte hain aur agent uske base par `args` parameter ke liye data generate karta hai, toh `RunContextWrapper[Any]` ka schema define nahi kiya gaya phir bhi agent ya LLM isko kaise samajhta hai aur `ctx` parameter mein value kaise pass karta hai. Main isko aapke code aur OpenAI Agents SDK ke context mein simple aur focused tareeke se samjhaunga, sath hi documentation ([https://openai.github.io/openai-agents-python/tools/#custom-function-tools](https://openai.github.io/openai-agents-python/tools/#custom-function-tools)) se relevant information include karunga.

---

### **1. Code Recap aur Sawal ka Context**
Aapka code yeh hai:

```python
from typing import Any
from pydantic import BaseModel
from agents import RunContextWrapper, FunctionTool

def do_some_work(data: str) -> str:
    return "done"

class FunctionArgs(BaseModel):
    username: str
    age: int

async def run_function(ctx: RunContextWrapper[Any], args: str) -> str:
    parsed = FunctionArgs.model_validate_json(args)
    return do_some_work(data=f"{parsed.username} is {parsed.age} years old")

tool = FunctionTool(
    name="process_user",
    description="Processes extracted user data",
    params_json_schema=FunctionArgs.model_json_schema(),
    on_invoke_tool=run_function,
)
```

- **Aapka Sawal**:
  - `FunctionArgs` ka JSON schema (`model_json_schema()`) agent ko batata hai ke `args` mein kaunsa data aayega (username aur age). LLM is schema ke base par input se data nikaalta hai aur `args` mein pass karta hai.
  - Lekin `RunContextWrapper[Any]` ka koi schema define nahi kiya gaya, toh agent ya LLM kaise jaanta hai ke `ctx` parameter mein kya pass karna hai? Aur yeh kaise kaam karta hai?

---

### **2. `RunContextWrapper[Any]` Kya Hai?**

- **`RunContextWrapper[Any]`** ek class hai jo OpenAI Agents SDK ke andar define hoti hai. Yeh agent ke execution context ko represent karta hai, yani agent ke current state aur run-time information ko hold karta hai.
- Ismein typically aapka custom context (jaise `MyCustomContext`) hota hai, jo `context.context` ke roop mein accessible hota hai, sath hi aur meta-data (jaise run ka state ya configuration).

- **Key Point**: `RunContextWrapper` ka schema explicitly define karne ki zarurat nahi hoti kyunki yeh SDK ka internal component hai, aur SDK khud isko manage karta hai.

---

### **3. Agent Ko `ctx` Mein Kya Pass Karna Hai Kaise Pata Chalta Hai?**

Agent ya LLM ko `RunContextWrapper` ke liye schema define karne ki zarurat nahi hoti kyunki:

1. **SDK Ka Internal Management**:
   - Jab aap `Runner.run` ya `Runner.run_sync` call karte hain aur `context` parameter mein apna custom context (jaise `my_context = MyCustomContext(name="Zain", age=21)`) pass karte hain, SDK is context ko `RunContextWrapper` mein wrap karta hai:
     ```python
     context_wrapper = RunContextWrapper(context=my_context)
     ```
   - Yeh `context_wrapper` ek `RunContextWrapper[Any]` object hota hai, jismein aapka `my_context` `context.context` ke roop mein store hota hai.

2. **Agent Ka Role**:
   - Jab agent `FunctionTool` (jaise `process_user`) ko invoke karta hai, SDK `run_function` ko call karta hai aur `ctx` parameter mein yeh `RunContextWrapper` object automatically pass karta hai.
   - Yeh process SDK ke internal logic ka hissa hai aur tool definition mein explicitly define nahi karna padta.

3. **Schema Ki Zarurat Nahi**:
   - `FunctionArgs` ke liye schema (`model_json_schema`) isliye define kiya jata hai kyunki `args` user input ya LLM ke decision ke base par dynamically generate hota hai. LLM ko batana padta hai ke `args` mein kaunsa data valid hai (jaise `username` aur `age`).
   - Lekin `RunContextWrapper` SDK ka ek standard component hai, jo har tool invocation mein automatically provide kiya jata hai. Iska structure SDK ke andar pre-defined hota hai, isliye iska schema tool definition mein dena nahi padta.

4. **LLM Ka Role**:
   - LLM directly `RunContextWrapper` ke content ko nahi dekhta. Woh sirf `args` ke liye JSON schema (`FunctionArgs.model_json_schema()`) dekhta hai taake input data generate kar sake.
   - `ctx` ko SDK handle karta hai, aur yeh `run_function` ke liye as a context pass hota hai taake aap ismein se additional information (jaise `context.context.name` ya `context.context.age`) access kar saken agar chahiye.

---

### **4. Backend Working Kaise Hoti Hai?**

Chaliye step-by-step dekhte hain ke yeh kaise kaam karta hai:

1. **Agent Initialization**:
   - Aapne `Agent` banaya hai aur usmein `tools=[process_user]` diya hai. `process_user` tool ka schema `FunctionArgs.model_json_schema()` se aata hai.

2. **Runner Call**:
   - Jab aap `Runner.run_sync(agent, input, context=my_context)` call karte hain, SDK yeh karta hai:
     - `my_context` ko `RunContextWrapper` mein wrap karta hai: `ctx = RunContextWrapper(context=my_context)`.
     - Input (jaise "Process user data for Ali, age 25") ko LLM ke pass bhejta hai.

3. **LLM Aur Tool Selection**:
   - LLM input aur `params_json_schema` ko dekhta hai aur decide karta hai ke `process_user` tool call karna hai.
   - LLM schema ke mutabiq `args` banata hai, jaise `{"username": "Ali", "age": 25}"`.

4. **Tool Invocation**:
   - SDK `run_function` ko call karta hai aur do parameters pass karta hai:
     - `ctx`: Yeh `RunContextWrapper` object hai jismein `my_context` wrapped hota hai.
     - `args`: Yeh JSON string hai jo LLM ne schema ke base par banayi hai.

5. **Run Function Processing**:
   - `run_function` mein `args` ko `FunctionArgs.model_validate_json(args)` se parse kiya jata hai, jo `parsed` object banata hai (`parsed.username`, `parsed.age`).
   - `ctx` ka is code mein direct istemal nahi hai, lekin agar aap chahen toh `ctx.context` se `my_context` ke fields (jaise `name`, `age`) access kar sakte hain.

6. **Output**:
   - `run_function` `do_some_work` ko call karta hai aur `"done"` return karta hai, jo agent ke output ka hissa banta hai.

---

### **5. `args` aur `ctx` Mein Difference**

- **`args`**:
  - Yeh dynamic data hai jo LLM input aur schema (`FunctionArgs.model_json_schema()`) ke base par generate karta hai.
  - Schema isliye zaroori hai kyunki LLM ko pata hona chahiye ke tool ke liye kaunsa data valid hai.
  - Example: `args = '{"username": "Ali", "age": 25}'`.

- **`ctx: RunContextWrapper[Any]`**:
  - Yeh SDK ka internal object hai jo agent ke execution context ko hold karta hai.
  - Iska schema define karne ki zarurat nahi kyunki SDK khud isko banata aur pass karta hai.
  - Yeh har tool invocation mein automatically provide hota hai aur ismein aapka custom context (`my_context`) hota hai.

---

### **6. `.model_json_schema` aur `.model_validate_json` Ka Connection**

- **`.model_json_schema()`**:
  - Yeh `FunctionArgs` ka JSON schema banata hai, jo agent ko batata hai ke `args` mein kaunsa data aayega.
  - Yeh tool definition mein `params_json_schema` ke roop mein pass hota hai, taake LLM input se sahi data generate kar sake.

- **`.model_validate_json()`**:
  - Yeh `args` (JSON string) ko parse aur validate karta hai taake `FunctionArgs` object bane.
  - Yeh ensure karta hai ke `args` schema ke mutabiq hai.

**Connection with `ctx`**:
- `ctx` ke liye aisa koi schema nahi banaya jata kyunki yeh SDK ka internal part hai. SDK khud `RunContextWrapper` banata hai jab aap `Runner.run` ya `Runner.run_sync` call karte hain aur `context=my_context` pass karte hain.

---

### **Summary**

- **Agent Ko `args` Kaise Pata Chalta Hai?**  
  `FunctionArgs.model_json_schema()` se bana JSON schema agent ko batata hai ke `args` mein kaunsa data aayega (jaise `username: str`, `age: int`). LLM is schema ke base par input se JSON string banata hai aur `args` ke roop mein pass karta hai.

- **Agent Ko `ctx` Kaise Pata Chalta Hai?**  
  `RunContextWrapper` ka schema define karne ki zarurat nahi kyunki SDK isko internally manage karta hai. Jab aap `Runner.run_sync` mein `context=my_context` pass karte hain, SDK `RunContextWrapper` banata hai aur `run_function` ke `ctx` parameter mein automatically pass karta hai.

- **Backend Working**:
  - `args`: LLM schema ke base par input se JSON string banata hai.
  - `ctx`: SDK `my_context` ko `RunContextWrapper` mein wrap karta hai aur pass karta hai.
  - `run_function` `args` ko parse karta hai aur `ctx` se additional context access kar sakta hai.


---------------
---------------

## **12. on_invoke_tool: Callable[[RunContextWrapper[Any], str], Awaitable[Any]] breakdown:**


1. **`Callable`**  
   - **Matlab**: Yeh ek aisa function ya method hona chahiye jis ko hum call kar sakein. Yani, yeh batata hai ke `on_invoke_tool` ek function hoga jo chalaya ja sakega.  
   - **Example**: Jaise `print()` ya koi custom function `my_function()` jo aap define karte hain.

2. **`[RunContextWrapper[Any], str]`**  
   - **Matlab**: Yeh batata hai ke function do cheezein (parameters) lega:  
     - **Pehla parameter**: `RunContextWrapper[Any]` type ka hoga. Yeh ek object hai jo agent ke current state ya context ko represent karta hai. `Any` ka matlab hai ke is ke andar koi bhi type ka data ho sakta hai.  
     - **Doosra parameter**: `str` type ka hoga, yani ek string. Yeh typically tool ke liye input data hota hai, jaise JSON string mein diya gaya data.  
   - **Simple language mein**: Function ko do cheezein chahiye – ek context (agent ki halat) aur ek string (input).  

3. **`Awaitable[Any]`**  
   - **Matlab**: Yeh batata hai ke function jo result return karega, woh `Awaitable` hoga. Yani, is ka result turant nahi milega, balki is ke liye wait karna padega. Yeh tab hi possible hai jab function asynchronous ho.  
   - **Async ka matlab**: Ek asynchronous function woh hota hai jo background mein kaam karta hai aur jab tak woh complete na ho, us ke result ke liye `await` keyword use karna padta hai.  
   - **`Any` ka matlab**: Return value kisi bhi type ka ho sakta hai – string, number, object, kuch bhi.  

### Ek Simple Example
Maano aap ke paas yeh function hai:
```python
async def my_tool(ctx: RunContextWrapper[Any], args: str) -> str:
    return "Hello, " + args
```
- Yeh `async` function hai, toh yeh `Awaitable` hai.  
- Yeh `ctx` (context) aur `args` (string input) leta hai.  
- Result mein ek string return karta hai, jo `Any` ke under aata hai.

### Choti si Summary
- **`on_invoke_tool`**: Ek function hai jo tool chalane ke liye call hota hai.  
- **Leta hai**: Do cheezein – `ctx` (context) aur `args` (string input).  
- **Deta hai**: Ek `Awaitable` result, jo asynchronous hota hai aur kisi bhi type ka ho sakta hai.

-------------


# **13. Understanding model_config=ConfigDict(extra="...") setting in Pydantic baseModel class:**

Main aapke explanation ko aur behtar banata hoon aur `extra="allow"` aur `extra="ignore"` ke concepts ko bhi simple tareeke se add karta hoon. Yeh explanation Pydantic library ke context mein hai aur aapke code ke `FunctionArgs` class ke saath connect karta hai.

---

### **Kya Hai `model_config=ConfigDict(extra="...")`?**

- Yeh ek Pydantic setting hai jo `BaseModel` class ke andar use hoti hai (jaise aapke `FunctionArgs` class mein).
- `ConfigDict` ek configuration dictionary hai jo Pydantic model ke behavior ko control karta hai, khas tor par yeh ke model input data ke extra fields ko kaise handle karega.
- `extra` parameter ke teen possible values hain: `"forbid"`, `"allow"`, aur `"ignore"`. Yeh teeno alag-alag tareeke se extra fields ko handle karte hain.

---

### **Simple Tareeke Se Samjhein**

- **Imagine**: Aapke paas ek form hai jismein sirf `username` aur `age` likhne ki jagah hai. Agar koi extra field (jaise `email`) bhi daal deta hai, toh `extra` setting decide karti hai ke model us extra field ke saath kya karega.

- **Teen Options for `extra`**:
  1. **`extra="forbid"`**:
     - **Matlab**: Model sirf woh fields accept karega jo define kiye gaye hain (jaise `username` aur `age`). Agar koi extra field diya gaya (jaise `email`), toh model error dega.
     - **Example**: `{"username": "Ali", "age": 25, "email": "ali@example.com"}` → Error, kyunki `email` define nahi hai.
     - **Kab Use Karein**: Jab aap strict validation chahte hain aur koi bhi extra data allow nahi karna chahte.

  2. **`extra="allow"`**:
     - **Matlab**: Model defined fields (`username`, `age`) ke saath-saath extra fields ko bhi accept karega aur unko store kar lega.
     - **Example**: `{"username": "Ali", "age": 25, "email": "ali@example.com"}` → Kaam karega, aur `email` bhi model mein store ho jayega (lekin `FunctionArgs` class mein `email` field define nahi hai, toh yeh dynamically store hota hai).
     - **Kab Use Karein**: Jab aapko flexible model chahiye jo extra data ko bhi handle kar sake.

  3. **`extra="ignore"`**:
     - **Matlab**: Model defined fields (`username`, `age`) ko accept karega, lekin extra fields (jaise `email`) ko ignore kar dega (na store karega, na error dega).
     - **Example**: `{"username": "Ali", "age": 25, "email": "ali@example.com"}` → Kaam karega, lekin `email` discard ho jayega.
     - **Kab Use Karein**: Jab aapko extra data chahiye nahi, lekin error bhi nahi chahiye.

---

### **Code Context Mein**

Aapka code yeh hai:

```python
from pydantic import BaseModel, ConfigDict

class FunctionArgs(BaseModel):
    username: str
    age: int
    model_config = ConfigDict(extra="forbid")
```

- **Yeh Kya Karta Hai?**  
  Jab aap `FunctionArgs.model_validate_json()` use karte hain, toh yeh input JSON string ko check karta hai:
  - **Agar `extra="forbid"`**: Sirf `username` aur `age` accept honge. Extra field (jaise `email`) hone par error aayega.
    - Valid: `{"username": "Ali", "age": 25}` → Kaam karega.
    - Invalid: `{"username": "Ali", "age": 25, "email": "ali@example.com"}` → Error dega.
  - **Agar `extra="allow"`**: Extra fields bhi accept honge aur store ho jayenge (lekin `FunctionArgs` object mein sirf `username` aur `age` directly accessible honge).
    - Valid: `{"username": "Ali", "age": 25, "email": "ali@example.com"}` → Kaam karega, `email` dynamically store hoga.
  - **Agar `extra="ignore"`**: Extra fields ignore ho jayenge, sirf `username` aur `age` process honge.
    - Valid: `{"username": "Ali", "age": 25, "email": "ali@example.com"}` → Kaam karega, `email` discard ho jayega.

---

### **Kyun Use Hota Hai?**

- **Data Validation**: Yeh galat ya unexpected data se bachata hai (`forbid`) ya flexibility deta hai (`allow`, `ignore`).
- **Strictness vs Flexibility**:
  - `"forbid"`: Strict validation ke liye, jab aapko sirf defined fields chahiye.
  - `"allow"`: Flexible validation ke liye, jab extra fields bhi store karne hain.
  - `"ignore"`: Jab extra fields ko nazarandaaz karna hai bina error ke.
- **Error Handling**: `forbid` bugs ko pehle pakadta hai, jabki `allow` aur `ignore` zyada lenient hote hain.

---

### **Example for Clarity**

```python
from pydantic import BaseModel, ConfigDict

# Case 1: extra="forbid"
class FunctionArgs(BaseModel):
    username: str
    age: int
    model_config = ConfigDict(extra="forbid")

# Case 2: extra="allow"
class FunctionArgsAllow(BaseModel):
    username: str
    age: int
    model_config = ConfigDict(extra="allow")

# Case 3: extra="ignore"
class FunctionArgsIgnore(BaseModel):
    username: str
    age: int
    model_config = ConfigDict(extra="ignore")

# Testing
json_data = '{"username": "Ali", "age": 25, "email": "ali@example.com"}'

# forbid
try:
    result = FunctionArgs.model_validate_json(json_data)
    print("Forbid:", result)  # Error: Extra inputs are not permitted
except Exception as e:
    print("Forbid Error:", str(e))

# allow
result = FunctionArgsAllow.model_validate_json(json_data)
print("Allow:", result)  # Works, email is stored dynamically
print("Allow Extra:", result.__dict__)  # Shows {'username': 'Ali', 'age': 25, 'email': 'ali@example.com'}

# ignore
result = FunctionArgsIgnore.model_validate_json(json_data)
print("Ignore:", result)  # Works, email is ignored
print("Ignore Extra:", result.__dict__)  # Shows {'username': 'Ali', 'age': 25}
```

---

### **Summary**

- `model_config=ConfigDict(extra="forbid")`:
  - Sirf defined fields (`username`, `age`) accept karta hai.
  - Extra fields (jaise `email`) par error deta hai.
  - Strict validation ke liye use hota hai.

- `model_config=ConfigDict(extra="allow")`:
  - Defined fields ke saath extra fields bhi accept aur store karta hai.
  - Flexible validation ke liye use hota hai.

- `model_config=ConfigDict(extra="ignore")`:
  - Defined fields accept karta hai, extra fields ko ignore karta hai.
  - Jab extra fields ko discard karna ho, bina error ke.

- Yeh teeno settings tool ke input ko validate aur process karne ke liye use hoti hain taake code predictable aur safe rahe.

------------


## **14. Tool file Code behind the scene:**

Chalo, main tumhe `@function_tool` decorator ke do overloads ka difference asaan alfaaz mein samjhaata hoon. Yeh code ek Python function ko `FunctionTool` mein badalne ke liye hai, jo OpenAI Agents SDK mein use hota hai. Iska matlab hai ki yeh decorator function ko ek tool bana deta hai, aur isko do tarah se use kiya ja sakta hai. In dono tarah ke liye alag-alag `@overload` banaye gaye hain. Ab inko step-by-step samjhte hain.

---

### **Pehla Overload: Jab `@function_tool` Bina Parentheses Ke Use Hota Hai**

**Code:**
```python
@overload
def function_tool(
    func: ToolFunction[...],
    *,
    name_override: str | None = None,
    description_override: str | None = None,
    docstring_style: DocstringStyle | None = None,
    use_docstring_info: bool = True,
    failure_error_function: ToolErrorFunction | None = None,
    strict_mode: bool = True,
) -> FunctionTool:
    """Overload for usage as @function_tool (no parentheses)."""
    ...
```

**Iska Matlab:**
- Yeh wala overload tab kaam aata hai jab tum decorator ko seedha function ke upar lagaate ho, bina koi extra settings (arguments) diye. Matlab, sirf `@function_tool` likha aur function ke saath use kiya.
  
**Example:**
```python
@function_tool
def add(a: int, b: int) -> int:
    return a + b
```
- Isme decorator direct `add` function pe laga hai, aur yeh ek `FunctionTool` bana deta hai bina kisi customization ke.

**Key Points:**
- **Parameters:**
  - `func`: Yeh woh function hai jo decorate ho raha hai (jaise `add`).
  - Baaki cheezein jaise `name_override`, `description_override`, etc., optional hain aur agar tum inko nahi badalte, toh default values use hoti hain.
- **Return Type:** Yeh direct ek `FunctionTool` return karta hai. Yani, function ko tool mein badal ke turant de deta hai.

**Kab Use Karna:**
- Jab tumhe koi extra settings change nahi karni, aur bas default tareeke se function ko tool banana hai.

---

### **Doosra Overload: Jab `@function_tool(...)` Parentheses Ke Saath Use Hota Hai**

**Code:**
```python
@overload
def function_tool(
    *,
    name_override: str | None = None,
    description_override: str | None = None,
    docstring_style: DocstringStyle | None = None,
    use_docstring_info: bool = True,
    failure_error_function: ToolErrorFunction | None = None,
    strict_mode: bool = True,
) -> Callable[[ToolFunction[...]], FunctionTool]:
    """Overload for usage as @function_tool(...)."""
    ...
```

**Iska Matlab:**
- Yeh overload tab kaam aata hai jab tum decorator ko customize karna chahte ho, yani usme arguments daal ke use karna chahte ho.

**Example:**
```python
@function_tool(name_override="custom_add", strict_mode=False)
def add(a: int, b: int) -> int:
    return a + b
```
- Yahan `@function_tool(name_override="custom_add", strict_mode=False)` likha hai, jo decorator ko batata hai ki tool ka naam "custom_add" rakho aur strict mode off kar do.

**Key Points:**
- **Parameters:**
  - Yahan `func` nahi hai kyunki yeh overload pehle ek decorator banata hai, jo baad mein function pe lagega.
  - Tum `name_override`, `strict_mode`, etc., jaise arguments daal sakte ho apni marzi se.
- **Return Type:** Yeh ek `Callable` return karta hai, yani ek aisa function jo baad mein asli function (jaise `add`) ko lega aur usko `FunctionTool` mein badlega.

**Kab Use Karna:**
- Jab tumhe tool banate waqt kuch settings change karni hon, jaise naam badalna ya strict mode on/off karna.

---

### **Asli Implementation: Yeh Kaise Kaam Karta Hai**

**Code:**
```python
def function_tool(
    func: ToolFunction[...] | None = None,
    *,
    name_override: str | None = None,
    description_override: str | None = None,
    docstring_style: DocstringStyle | None = None,
    use_docstring_info: bool = True,
    failure_error_function: ToolErrorFunction | None = default_tool_error_function,
    strict_mode: bool = True,
) -> FunctionTool | Callable[[ToolFunction[...]], FunctionTool]:
    ...
```

**Iska Kaam:**
- Yeh asli wala `function_tool` hai jo upar ke dono overloads ko handle karta hai.
- **Do Tarah Se Kaam Karta Hai:**
  1. **Agar `func` Diya Gaya (Bina Parentheses):** Agar tumne `@function_tool` seedha use kiya, toh yeh direct `func` ko `FunctionTool` mein badal deta hai.
  2. **Agar `func` Nahi Diya (Parentheses Ke Saath):** Agar tumne `@function_tool(name_override="xyz")` likha, toh yeh ek decorator function return karta hai, jo baad mein tumhare function pe lagega.

**Example Samajh Lo:**
- `@function_tool`: Yeh seedha kaam karta hai, jaise ek button dabaya aur tool ban gaya.
- `@function_tool(name_override="xyz")`: Yeh pehle ek chhota sa function banata hai, jo baad mein tumhare function ko tool mein badlega jab tum uspe lagao.

---

### **Dono Mein Main Difference Kya Hai?**

| **Baatein**            | **Pehla Overload (Bina Parentheses)**             | **Doosra Overload (Parentheses Ke Saath)**       |
|-------------------------|--------------------------------------------------|-------------------------------------------------|
| **Usage**              | Seedha `@function_tool` likha jaata hai          | `@function_tool(name_override="xyz")` likha jaata hai |
| **Customization**      | Default settings use hoti hain                  | Tum apne hisaab se settings badal sakte ho      |
| **Return Type**        | Direct `FunctionTool` milta hai                 | Pehle ek `Callable` (decorator) milta hai       |
| **Parameters**         | `func` zaruri hai kyunki function seedha lega   | `func` nahi hai, baad mein function lega        |

---

### **Yeh Overloads Kyun Banaye?**

- **Samajh Aasaan Ho:** `@overload` se yeh clear ho jaata hai ki `function_tool` ko do tarah se use kar sakte ho.
- **Galti Na Ho:** Type checkers (jaise `mypy`) ko pata hota hai ki kaunsa tareeka sahi hai, toh code likhte waqt galti hone ki sambhavna kam hoti hai.

---

### **Aasan Alfaaz Mein Conclusion**

- **Pehla Overload:** Jab seedha kaam karna ho, bina kisi setting change kiye, toh bas `@function_tool` laga do.
- **Doosra Overload:** Jab thodi customization chahiye, jaise naam badalna ya strict mode off karna, toh `@function_tool(...)` use karo.

-------------------------

Chalo, main tumhe yeh code bilkul aasan zabaan mein samjhaata hoon, kyunki tum Python mein beginner ho. Yeh code thoda tricky lag sakta hai, lekin main isko step-by-step break karke saaf kar dunga taake tumhe bilkul samajh aaye.

---

### **Yeh Code Kya Hai?**
Yeh code ek **decorator** ke baare mein hai jo Python mein use hota hai. Decorator ka naam hai `@function_tool`. Yeh decorator do tarah se kaam kar sakta hai:
1. **Bina settings ke**: Jaise `@function_tool` seedha likha jaye.
2. **Settings ke saath**: Jaise `@function_tool(name_override="xyz")`.

Iska maqsad yeh decide karna hai ke decorator ka use kaise hua hai aur uske hisaab se function ko handle karna hai.

---

### **Code Dekho**
Yeh hai woh code jo tumhe samajh nahi aaya:

```python
# If func is actually a callable, we were used as @function_tool with no parentheses
if callable(func):
    return _create_function_tool(func)

# Otherwise, we were used as @function_tool(...), so return a decorator
def decorator(real_func: ToolFunction[...]) -> FunctionTool:
    return _create_function_tool(real_func)

return decorator
```

Ab isko ek-ek line samajhte hain.

---

### **Line-by-Line Samajh**

#### **1. `if callable(func):`**
- **`callable(func)`**: Yeh check karta hai ke `func` ek function hai ya nahi, yani kya isko call kiya ja sakta hai (jaise `add()`).
- **Agar `func` callable hai**: Iska matlab hai ke decorator ko bina kisi settings ke use kiya gaya hai. Matlab, tumne seedha `@function_tool` likha hai kisi function ke upar.
- **Example**:
  ```python
  @function_tool
  def add(a, b):
      return a + b
  ```
  Yahan `func` hai `add`, jo ek function hai aur callable hai.
- **Kya Hota Hai**: Agar yeh condition true hai, toh yeh `_create_function_tool(func)` chalata hai. Yeh ek function hai jo `func` (jaise `add`) ko ek `FunctionTool` mein badal deta hai aur return kar deta hai.

---

#### **2. Agar `func` Callable Nahi Hai (Else Part)**
- **Agar `func` callable nahi hai**: Iska matlab hai ke decorator ko settings ke saath use kiya gaya hai, jaise `@function_tool(name_override="xyz")`.
- **Example**:
  ```python
  @function_tool(name_override="xyz")
  def add(a, b):
      return a + b
  ```
  Yahan `func` seedha function nahi hai, balki settings (jaise `name_override`) pass ki gayi hain.
- **Kya Hota Hai**: Is case mein, code ek naya **decorator** banata hai jo baad mein asli function (jaise `add`) ko handle karega.

---

#### **3. `def decorator(real_func: ToolFunction[...]) -> FunctionTool:`**
- **Yeh Kya Hai**: Yeh ek chhota sa function (decorator) hai jo `real_func` naam ka asli function leta hai (jaise `add`) aur usko `_create_function_tool(real_func)` ke zariye `FunctionTool` mein badal deta hai.
- **Kaam**: Yeh settings wala decorator banata hai, jo function ko customize karke tool mein badalne ke liye use hota hai.

---

#### **4. `return decorator`**
- **Matlab**: Agar settings ke saath decorator use hua hai, toh yeh `decorator` function return karta hai. Yeh decorator baad mein asli function pe lagaya jayega.
- **Kaise Kaam Karta Hai**:
  - Pehle `@function_tool(name_override="xyz")` ek decorator banata hai.
  - Phir jab tum `def add(a, b):` likhte ho, yeh us decorator ko `add` pe lagata hai, aur `_create_function_tool(add)` chalata hai.

---

### **Aasan Bhasha Mein**
- **Agar Tum `@function_tool` Likhte Ho**:
  - Yeh check karta hai ke `func` ek function hai, aur seedha usko `FunctionTool` mein badal deta hai.
- **Agar Tum `@function_tool(name_override="xyz")` Likhte Ho**:
  - Yeh pehle ek chhota function (decorator) banata hai.
  - Phir jab tum apna function likhte ho, yeh usko customize karke `FunctionTool` mein badalta hai.

---

### **Examples Se Samajh Lo**

#### **Example 1: Bina Settings Ke**
```python
@function_tool
def add(a, b):
    return a + b
```
- Yahan `func = add`, jo callable hai.
- Toh yeh direct `_create_function_tool(add)` chalata hai aur `FunctionTool` banata hai.

#### **Example 2: Settings Ke Saath**
```python
@function_tool(name_override="xyz")
def add(a, b):
    return a + b
```
- Yahan pehle `function_tool(name_override="xyz")` chalega, jo ek decorator banayega.
- Phir yeh decorator `add` function pe lagega, aur `_create_function_tool(add)` chalega.

---

### **Kyun Aisa Kiya?**
Yeh code isliye likha gaya hai taake decorator ko **do tarah se use kar sakein**:
1. Simple tareeke se (bina settings ke).
2. Customize karke (settings ke saath).

---

### **Choti Si Baat**
- **`if` Wala Part**: Bina settings ke seedha tool banata hai.
- **`else` Wala Part**: Settings ke saath ek decorator banata hai, jo baad mein function ko tool mein badalta hai.

